# Your Title Here

**Name(s)**: (your name(s) here)

**Website Link**: (your website link)

In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
pd.options.plotting.backend = 'plotly'


# from dsc80_utils import * # Feel free to uncomment and use this.

In [15]:
import plotly.io as pio
pio.renderers.default = 'jupyterlab'

## Step 1: Introduction

1. We picked outage dataset because it is rich in information as well as something we have a bit of knowledge in. Also because it is interesting to see the result and analyze what causes power outages
2. plotly vis
3. hypotheses
4. prediction?

We will be answering the following question:  
what characteristic of power outages causes the most economic loss?

## Step 2: Data Cleaning and Exploratory Data Analysis

In [16]:
outage = pd.read_csv('data/power_outage.csv')
outage.columns

Index(['OBS', 'YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
    

In [17]:

pd.set_option('display.max_columns', None)
outage = outage[['U.S._STATE', 'POSTAL.CODE',
       'NERC.REGION', 'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE','POPULATION']]

In the following section we deal with missing data

In [18]:
# outage.dropna(inplace = True, how='any',subset=['OUTAGE.RESTORATION.DATE','RES.PRICE','CLIMATE.REGION'])

In the following we edit specific columns like data to make it easier to answer the main question:  
combining outage date and time

In [19]:

outage['OUTAGE_START']=pd.to_datetime(outage['OUTAGE.START.DATE']+' '+outage['OUTAGE.START.TIME'])
outage['OUTAGE_END']=pd.to_datetime(outage['OUTAGE.RESTORATION.DATE']+' '+outage['OUTAGE.RESTORATION.TIME'])
outage.drop(inplace=True,columns=['OUTAGE.START.DATE','OUTAGE.RESTORATION.DATE','OUTAGE.START.TIME','OUTAGE.RESTORATION.TIME'])

/var/folders/s9/b35x2lvd0zl66546s25wwsjc0000gn/T/ipykernel_37700/4024656936.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/s9/b35x2lvd0zl66546s25wwsjc0000gn/T/ipykernel_37700/4024656936.py:2: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



In [20]:
outage.isna().sum()

U.S._STATE                  0
POSTAL.CODE                 0
NERC.REGION                 0
CLIMATE.REGION              6
ANOMALY.LEVEL               9
CLIMATE.CATEGORY            9
CAUSE.CATEGORY              0
CAUSE.CATEGORY.DETAIL     471
HURRICANE.NAMES          1462
OUTAGE.DURATION            58
DEMAND.LOSS.MW            705
CUSTOMERS.AFFECTED        443
RES.PRICE                  22
COM.PRICE                  22
IND.PRICE                  22
TOTAL.PRICE                22
POPULATION                  0
OUTAGE_START                9
OUTAGE_END                 58
dtype: int64

UNIVARIATE ANALYSIS

In [21]:
fig1 = px.histogram(outage, x='POPULATION')
fig2 = px.histogram(outage, x='OUTAGE.DURATION')
climate_ct_group = outage.groupby('CLIMATE.CATEGORY')['OUTAGE.DURATION'].count()
fig3 = px.bar(climate_ct_group,labels={'CLIMATE.CATEGORY':'Climate','value':'Count'})

fig1.show()
fig2.show()
fig3.show()

BIVARIATE ANALYSIS

In [22]:
fig1 = px.scatter(outage, x='POPULATION', y='OUTAGE.DURATION')
fig2 = px.scatter(outage, x='ANOMALY.LEVEL', y='OUTAGE.DURATION',title='Scatter Plot: ANOMALYLLYRL vs. OUTAGE DURATION',
                  labels={'ANOMALY.LEVEL': 'Anomaly Level', 'OUTAGE.DURATION': 'Outage Duration (hours)'})
fig3 = px.box(outage, x='CAUSE.CATEGORY', y='OUTAGE.DURATION',
              title='Box Plot: OUTAGE DURATION by CAUSE CATEGORY',
              labels={'CAUSE.CATEGORY': 'Cause Category', 'OUTAGE.DURATION': 'Outage Duration (hours)'})
fig4 = px.histogram(outage, x='ANOMALY.LEVEL', color='CLIMATE.CATEGORY', barmode='overlay',
                    title='Conditional Distribution: ANOMALYLLYRL by CLIMATE CATEGORY',
                    labels={'ANOMALY.LEVEL': 'Anomaly Level', 'count': 'Frequency'})
climate_avg_duration = outage.groupby('CLIMATE.CATEGORY')['OUTAGE.DURATION'].mean().reset_index()
fig5 = px.bar(climate_avg_duration, x='CLIMATE.CATEGORY', y='OUTAGE.DURATION',
               title='Average Outage Duration by CLIMATE CATEGORY',
               
               labels={'CLIMATE.CATEGORY': 'Climate Category', 'OUTAGE.DURATION': 'Average Outage Duration (hours)'})




# Heatmap: Outage Count by CAUSE CATEGORY and CLIMATE CATEGORY
pivot_table = pd.pivot_table(outage, values='OUTAGE.DURATION', index='CAUSE.CATEGORY', columns='CLIMATE.CATEGORY', aggfunc='count')
fig6 = px.imshow(pivot_table, labels=dict(x="Climate Category", y="Cause Category", color="Outage Count"),
                 title='Outage Count by CAUSE CATEGORY and CLIMATE CATEGORY')


# Bar plot: Maximum Anomaly Level by STATE
state_max_anomaly = outage.groupby('U.S._STATE')['ANOMALY.LEVEL'].mean().reset_index()
fig7 = px.bar(state_max_anomaly, x='U.S._STATE', y='ANOMALY.LEVEL',
              title='Maximum Anomaly Level by STATE',
              labels={'U.S._STATE': 'State', 'ANOMALY.LEVEL': 'Maximum Anomaly Level'})
fig8 = px.scatter(outage, x='ANOMALY.LEVEL', y='OUTAGE.DURATION')
fig1.show()
fig2.show()
fig3.show()
fig4.show()
fig5.show()
fig6.show()
fig7.show()
fig8.show()


## Step 3: Assessment of Missingness

In [ ]:
def permutation_test(df, group_col, missing_col, n_permutations=1000):
    observed_diff = df.groupby(group_col)[missing_col].mean().diff().abs().sum()
    perm_diffs = []
    for _ in range(n_permutations):
        df_perm = df.copy()
        df_perm[missing_col] = np.random.permutation(df_perm[missing_col])
        perm_diff = df_perm.groupby(group_col)[missing_col].mean().diff().abs().sum()
        perm_diffs.append(perm_diff)
    p_value = (np.array(perm_diffs) >= observed_diff).mean()
    return p_value

In [37]:
permutation_test(outage, 'CUSTOMERS.AFFECTED', 'DEMAND.LOSS.MW', n_permutations=1000)

np.float64(0.565)

In [24]:
outage['CAUSE CATEGORY DI_MISSING'] = outage['CAUSE.CATEGORY.DETAIL'].isna().astype(int)
observed = outage.groupby('CLIMATE.CATEGORY')['CAUSE CATEGORY DI_MISSING'].mean()
p_value = permutation_test(outage, 'CLIMATE.CATEGORY', 'CAUSE CATEGORY DI_MISSING')
p_value

np.float64(0.001)

After doing the permutation test on climate category and cause category missingness, the pval came out to be 0.001  
Since pval<0.05, this means missingness is very likely to be dependent on climate category. Additonally, this means cause missingness is Missing at Random (MAR)

## Step 4: Hypothesis Testing

##### We want to test if outage duration is a factor for economic loss.  
Null Hypothesis: Outage duration has no effect on economic loss. The mean economic loss is the same for short and long outages.  
Alternative Hypothesis: Outage duration affects ECONOMIC.LOSS. The mean economic loss is higher for long outages compared to short outages.  
Test Statistic: difference in mean for economic loss. 

compute economic loss

In [25]:
outage

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,POPULATION,OUTAGE_START,OUTAGE_END,CAUSE CATEGORY DI_MISSING
0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,5348119,2011-07-01 17:00:00,2011-07-03 20:00:00,1
1,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,5457125,2014-05-11 18:38:00,2014-05-11 18:39:00,0
2,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,5310903,2010-10-26 20:00:00,2010-10-28 22:00:00,0
3,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,5380443,2012-06-19 04:30:00,2012-06-20 23:00:00,0
4,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,5489594,2015-07-18 02:00:00,2015-07-19 07:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,NaN,720.0,155.0,34500.0,8.41,7.80,6.20,7.56,685326,2011-12-06 08:00:00,2011-12-06 20:00:00,1
1530,North Dakota,ND,MRO,West North Central,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650.0,NaN,NaN,NaN,NaN,NaN,649422,NaT,NaT,0
1531,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,NaN,59.0,84.0,NaN,9.25,7.47,5.53,7.67,807067,2009-08-29 22:54:00,2009-08-29 23:53:00,1
1532,South Dakota,SD,MRO,West North Central,0.5,warm,islanding,NaN,NaN,181.0,373.0,NaN,9.25,7.47,5.53,7.67,807067,2009-08-29 11:00:00,2009-08-29 14:01:00,1


In [ ]:
def test_missingness(df, target_column):
    """
    Tests whether missing values in the target_column are MCAR or MAR.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        target_column (str): The column to test for missingness.
    
    Returns:
        str: "MCAR" if missing values are completely at random, or
             a list of columns that are associated with the missingness of the target column (MAR).
    """
    # Create a binary column indicating if the value is missing in the target column
    df_c=df.copy()

    df_c['missing_indicator'] = df_c[target_column].isnull().astype(int)
    
    related_columns = []
    
    # Test association between missingness and other columns using heuristics
    for col in df.columns:
        if col != target_column and col != 'missing_indicator':
            # Drop rows where both columns are missing
            subset = df_c[[col, 'missing_indicator']].dropna()
            
            # Check if the column is numeric or categorical
            if pd.api.types.is_numeric_dtype(df_c[col]):
                # Compare means for rows with and without missing values
                with_missing = subset[subset['missing_indicator'] == 1][col]
                without_missing = subset[subset['missing_indicator'] == 0][col]
                
                # If the means differ significantly, consider it related
                mean_diff = abs(with_missing.mean() - without_missing.mean())
                threshold = 0.1 * without_missing.mean()  # 10% threshold
                if mean_diff > threshold:
                    related_columns.append(col)
            else:
                # For categorical columns, compare proportions
                with_missing_counts = subset[subset['missing_indicator'] == 1][col].value_counts(normalize=True)
                without_missing_counts = subset[subset['missing_indicator'] == 0][col].value_counts(normalize=True)
                
                # If proportions differ significantly, consider it related
                proportion_diff = (with_missing_counts - without_missing_counts).abs().sum()
                if proportion_diff > 0.1:  # Arbitrary threshold for significant difference
                    related_columns.append(col)
    
    
    # Return result
    if len(related_columns) == 0:
        return "MCAR"
    else:
        return related_columns

In [ ]:
def probabilistic_imputation(df, target_column, related_columns):
    """
    Imputes missing values in the target column using probabilistic imputation 
    based on related columns.
    
    Parameters:
        df (pd.DataFrame): The input DataFrame.
        target_column (str): The column with missing values to impute.
        related_columns (list): List of columns related to the target column's missingness.
    
    Returns:
        pd.DataFrame: The DataFrame with the missing values in the target column imputed.
    """
    # Ensure related_columns is a list
    if isinstance(related_columns, str) and related_columns == "MCAR":
        raise ValueError("Missingness is MCAR. Use simpler imputation methods like mean/median.")
    
    # Ensure the target column and related columns exist in the DataFrame
    if target_column not in df.columns or any(col not in df.columns for col in related_columns):
        raise ValueError("Target column or related columns are not present in the DataFrame.")
    
    # Verify that all columns are numeric
    if not pd.api.types.is_numeric_dtype(df[target_column]):
        raise TypeError(f"Target column '{target_column}' must be numeric.")
    for col in related_columns:
        if not pd.api.types.is_numeric_dtype(df[col]):
            raise TypeError(f"Related column '{col}' must be numeric.")
    
    # Split data into rows with and without missing values in the target column
    train_data = df[df[target_column].notnull()]
    test_data = df[df[target_column].isnull()]
    
    # Fit a simple linear regression model manually using NumPy
    X_train = train_data[related_columns].values  # Independent variables for training
    y_train = train_data[target_column].values    # Target variable for training
    
    # Add a bias term to X_train (intercept)
    X_train = np.hstack([np.ones((X_train.shape[0], 1)), X_train])
    
    # Compute regression coefficients using ordinary least squares
    coefficients = np.linalg.inv(X_train.T @ X_train) @ X_train.T @ y_train
    
    # Prepare test data (independent variables)
    X_test = test_data[related_columns].values
    X_test = np.hstack([np.ones((X_test.shape[0], 1)), X_test])  # Add bias term
    
    # Predict missing values
    predicted_values = X_test @ coefficients
    
    # Add stochastic noise based on the standard deviation of training residuals
    residuals = y_train - (X_train @ coefficients)
    noise_std = np.std(residuals)
    stochastic_noise = np.random.normal(0, noise_std, size=len(predicted_values))
    
    predicted_values += stochastic_noise
    
    # Replace missing values in the target column with predictions
    df.loc[df[target_column].isnull(), target_column] = predicted_values
    
    return df

In [50]:
test_missingness(outage, 'DEMAND.LOSS.MW')

['U.S._STATE',
 'POSTAL.CODE',
 'NERC.REGION',
 'CLIMATE.REGION',
 'ANOMALY.LEVEL',
 'CAUSE.CATEGORY',
 'CAUSE.CATEGORY.DETAIL',
 'HURRICANE.NAMES',
 'CUSTOMERS.AFFECTED',
 'POPULATION']

In [53]:
probabilistic_imputation(outage, 'DEMAND.LOSS.MW', ['U.S._STATE',
 'POSTAL.CODE',
 'NERC.REGION',
 'CLIMATE.REGION',
 'ANOMALY.LEVEL',
 'CAUSE.CATEGORY',
 'CAUSE.CATEGORY.DETAIL',
 'HURRICANE.NAMES',
 'CUSTOMERS.AFFECTED',
 'POPULATION'])

TypeError: can't multiply sequence by non-int of type 'float'

In [26]:
outage['ENERGY.LOSS.MWH'] = outage['DEMAND.LOSS.MW'] * outage['OUTAGE.DURATION']

# Step 3: Convert electricity price to dollars/MWh
outage['TOTAL.PRICE.DOLLARS.MWH'] = outage['TOTAL.PRICE'] / 100 * 1000  # Convert cents/kWh to dollars/MWh

# Step 4: Calculate economic loss
outage['ECONOMIC.LOSS'] = outage['ENERGY.LOSS.MWH'] * outage['TOTAL.PRICE.DOLLARS.MWH']

In [27]:
outage

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,POPULATION,OUTAGE_START,OUTAGE_END,CAUSE CATEGORY DI_MISSING,ENERGY.LOSS.MWH,TOTAL.PRICE.DOLLARS.MWH,ECONOMIC.LOSS
0,Minnesota,MN,MRO,East North Central,-0.3,normal,severe weather,NaN,NaN,3060.0,NaN,70000.0,11.60,9.18,6.81,9.28,5348119,2011-07-01 17:00:00,2011-07-03 20:00:00,1,NaN,92.8,NaN
1,Minnesota,MN,MRO,East North Central,-0.1,normal,intentional attack,vandalism,NaN,1.0,NaN,NaN,12.12,9.71,6.49,9.28,5457125,2014-05-11 18:38:00,2014-05-11 18:39:00,0,NaN,92.8,NaN
2,Minnesota,MN,MRO,East North Central,-1.5,cold,severe weather,heavy wind,NaN,3000.0,NaN,70000.0,10.87,8.19,6.07,8.15,5310903,2010-10-26 20:00:00,2010-10-28 22:00:00,0,NaN,81.5,NaN
3,Minnesota,MN,MRO,East North Central,-0.1,normal,severe weather,thunderstorm,NaN,2550.0,NaN,68200.0,11.79,9.25,6.71,9.19,5380443,2012-06-19 04:30:00,2012-06-20 23:00:00,0,NaN,91.9,NaN
4,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,5489594,2015-07-18 02:00:00,2015-07-19 07:00:00,1,435000.0,104.3,45370500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1529,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,NaN,720.0,155.0,34500.0,8.41,7.80,6.20,7.56,685326,2011-12-06 08:00:00,2011-12-06 20:00:00,1,111600.0,75.6,8436960.0
1530,North Dakota,ND,MRO,West North Central,NaN,NaN,fuel supply emergency,Coal,NaN,NaN,1650.0,NaN,NaN,NaN,NaN,NaN,649422,NaT,NaT,0,NaN,NaN,NaN
1531,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,NaN,59.0,84.0,NaN,9.25,7.47,5.53,7.67,807067,2009-08-29 22:54:00,2009-08-29 23:53:00,1,4956.0,76.7,380125.2
1532,South Dakota,SD,MRO,West North Central,0.5,warm,islanding,NaN,NaN,181.0,373.0,NaN,9.25,7.47,5.53,7.67,807067,2009-08-29 11:00:00,2009-08-29 14:01:00,1,67513.0,76.7,5178247.1


In [36]:
outage[outage['ECONOMIC.LOSS'].notna()]

,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,CAUSE.CATEGORY,CAUSE.CATEGORY.DETAIL,HURRICANE.NAMES,OUTAGE.DURATION,DEMAND.LOSS.MW,CUSTOMERS.AFFECTED,RES.PRICE,COM.PRICE,IND.PRICE,TOTAL.PRICE,POPULATION,OUTAGE_START,OUTAGE_END,CAUSE CATEGORY DI_MISSING,ENERGY.LOSS.MWH,TOTAL.PRICE.DOLLARS.MWH,ECONOMIC.LOSS
4,Minnesota,MN,MRO,East North Central,1.2,warm,severe weather,NaN,NaN,1740.0,250.0,250000.0,13.07,10.16,7.74,10.43,5489594,2015-07-18 02:00:00,2015-07-19 07:00:00,1,435000.0,104.3,45370500.0
7,Minnesota,MN,MRO,East North Central,0.2,normal,severe weather,thunderstorm,NaN,3960.0,75.0,300000.0,9.10,7.22,5.69,7.36,5119598,2005-06-08 04:00:00,2005-06-10 22:00:00,0,297000.0,73.6,21859200.0
8,Minnesota,MN,MRO,East North Central,0.6,warm,intentional attack,sabotage,NaN,155.0,20.0,5941.0,11.53,8.89,6.61,9.03,5489594,2015-03-16 07:31:00,2015-03-16 10:06:00,0,3100.0,90.3,279930.0
12,Minnesota,MN,MRO,East North Central,-0.3,normal,intentional attack,vandalism,NaN,0.0,0.0,0.0,11.60,9.18,6.81,9.28,5348119,2011-07-08 10:00:00,2011-07-08 10:00:00,0,0.0,92.8,0.0
13,Minnesota,MN,MRO,East North Central,-0.4,normal,intentional attack,vandalism,NaN,1322.0,0.0,0.0,11.23,8.91,6.46,8.70,5348119,2011-05-11 15:55:00,2011-05-12 13:57:00,0,0.0,87.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1524,Idaho,ID,WECC,Northwest,-0.1,normal,public appeal,NaN,NaN,1548.0,0.0,0.0,6.31,5.17,4.24,4.94,1363380,2003-06-15 15:12:00,2003-06-16 17:00:00,1,0.0,49.4,0.0
1526,Idaho,ID,WECC,Northwest,1.6,warm,intentional attack,sabotage,NaN,0.0,0.0,0.0,9.80,7.66,5.98,8.05,1680026,2016-03-08 00:00:00,2016-03-08 00:00:00,0,0.0,80.5,0.0
1529,North Dakota,ND,MRO,West North Central,-0.9,cold,public appeal,NaN,NaN,720.0,155.0,34500.0,8.41,7.80,6.20,7.56,685326,2011-12-06 08:00:00,2011-12-06 20:00:00,1,111600.0,75.6,8436960.0
1531,South Dakota,SD,RFC,West North Central,0.5,warm,islanding,NaN,NaN,59.0,84.0,NaN,9.25,7.47,5.53,7.67,807067,2009-08-29 22:54:00,2009-08-29 23:53:00,1,4956.0,76.7,380125.2


now for hypothesis testing

preperation

In [28]:
outage_clean = outage.dropna(subset=['OUTAGE.DURATION', 'ECONOMIC.LOSS'])

In [29]:
# Drop rows with missing values in these columns
outage_clean = outage.dropna(subset=['OUTAGE.DURATION', 'ECONOMIC.LOSS'])

# Define "short" and "long" outages based on the median duration
median_duration = outage_clean['OUTAGE.DURATION'].median()
outage_clean['OUTAGE.TYPE'] = outage_clean['OUTAGE.DURATION'].apply(lambda x: 'Long' if x > median_duration else 'Short')

/var/folders/s9/b35x2lvd0zl66546s25wwsjc0000gn/T/ipykernel_37700/563010878.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



testing  


In [30]:
# Observed difference in mean economic loss
observed_diff = (outage_clean[outage_clean['OUTAGE.TYPE'] == 'Long']['ECONOMIC.LOSS'].mean() -
                 outage_clean[outage_clean['OUTAGE.TYPE'] == 'Short']['ECONOMIC.LOSS'].mean())

# Permutation test
n_permutations = 1000
perm_diffs = []
for _ in range(n_permutations):
    # Shuffle the OUTAGE TYPE labels
    shuffled_labels = np.random.permutation(outage_clean['OUTAGE.TYPE'])
    # Calculate the difference in means for the shuffled data
    perm_diff = (outage_clean[shuffled_labels == 'Long']['ECONOMIC.LOSS'].mean() -
                 outage_clean[shuffled_labels == 'Short']['ECONOMIC.LOSS'].mean())
    perm_diffs.append(perm_diff)

# Calculate the p-value
p_value = (np.abs(np.array(perm_diffs)) >= np.abs(observed_diff)).mean()
print(f"Observed Difference: {observed_diff}")
print(f"P-value: {p_value}")

Observed Difference: 344043144.2550251
P-value: 0.0


In [31]:
fig = px.box(outage_clean, x='OUTAGE.TYPE', y='ECONOMIC.LOSS',
             title='Economic Loss by Outage Duration',
             labels={'OUTAGE.TYPE': 'Outage Duration', 'ECONOMIC.LOSS': 'Economic Loss'})
fig.show()

## Step 5: Framing a Prediction Problem

We want to predict economic loss. This will be a regression problem. 

In [32]:
# TODO

## Step 6: Baseline Model

In [33]:
# TODO

## Step 7: Final Model

In [34]:
# TODO

## Step 8: Fairness Analysis

In [35]:
# TODO